# Plotting predictions

This code plots the [distribution of prediction errors and confidence intervals for all counties](../outputs/ESM_9.tiff), predictions for the [present](../outputs/ESM_11.tiff) and [past](../outputs/Fig4.tiff) distribution of buckwheat, as well as the [suitability of locations with buckwheat overlaid on the overall suitability of environmental conditions across time](../outputs/Fig3.tiff). 

In [ ]:
### Load libraries
library(here)
library(rgeos)
library(geosphere)
library(rgdal)
library("ggplot2")
library("tidyr")
library(raster) # for crs
library(ggbeeswarm)
library(tmap)
library(sf)
library(stringr)
library(xlsx)

In [2]:
### Load utility functions and themes
source(here("R","getNicheOutline.R"))
source(here("R","theme_tmaps.R"))

In [3]:
### Define paths

# Define imputs paths:

# Admnistrative map of China
path2china<-here('raw_data','CHN_adm')

# Archaeological records location data
path2buckwheat<-here("data","ESM_2.csv") # Buckwheat's occurrence records
path2locations<-here("data","guedesbocinsky2018_crops_across_eurasia_translated.xlsx")

# Production and predictions data
path2production<-here('data','production','by_county')
path2predictions <- here("outputs","predictions")

In [4]:
# Define output paths:

path2outputs<-here("manuscript","Chapter6","Figs","Raster")

path2difference<-paste(path2outputs,"posterior_difference.png",sep="//")
path2intervals<-paste(path2outputs,"prediction_intervals.png",sep="//")

In [5]:
# Import data:
# Import the outer border of China
outer_border<-st_read(dsn=path2china,layer='CHN_adm0')
# Import the spatial polygons data frame of counties:
china<-readOGR(dsn=path2china,layer='CHN_adm3')
# Import production data

Reading layer `CHN_adm0' from data source `G:\My Drive\my_repositories\PhD\raw_data\CHN_adm' using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 70 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: 73.5577 ymin: 18.15931 xmax: 134.7739 ymax: 53.56086
geographic CRS: WGS 84
OGR data source with driver: ESRI Shapefile 
Source: "G:\My Drive\my_repositories\PhD\raw_data\CHN_adm", layer: "CHN_adm3"
with 2409 features
It has 13 fields
Integer64 fields read as strings:  ID_0 ID_1 ID_2 ID_3 


In [6]:
# List of crops in the analysis:

crops<-c("rice","millet","buckwheat","wheat","barley")

# Posterior predictive check

In [7]:
# Loop over crops to get their summaries for the predictive check:

for (crop in crops){
    print(crop)
    # Add the paht
    path2prod<-paste(path2production,"/",crop,".csv",sep="")
    path2post <-paste(path2predictions,"/",crop,"_posterior_predictions_summary.csv",sep="")

    # Read production data and posterior distribution
    posterior<-read.csv(path2post)
    prd<-read.csv(path2prod,row.names=1)

    # Get the observed values
    logArea<-prd$logProportionAreaCultivated

    # Get mean and confidence interval for posterior predictions
    mean<-posterior[,colnames(posterior)[grepl("log",colnames(posterior))&grepl("mean",colnames(posterior))]]
    q5<-posterior[,colnames(posterior)[grepl("log",colnames(posterior))&grepl("X5",colnames(posterior))]]
    q95<-posterior[,colnames(posterior)[grepl("log",colnames(posterior))&grepl("X95",colnames(posterior))]]

    # Put the values together in the dataframe
    df<-data.frame(mean,q5,q95,logArea)

    # Calculate the difference between predictions an observation
    df$difference<-df$mean - df$logArea
    df$interval<-df$q95 - df$q5

    # Add the name of the crop to the column names
    colnames(df)<-paste(crop,colnames(df),sep="_")
    
    # Bind data to spatial polygons
    china@data<-cbind(china@data,df)
    }

[1] "rice"
[1] "millet"
[1] "buckwheat"
[1] "wheat"
[1] "barley"


In [8]:
### Gather data by differenc and interval for plotting
difference<-gather(china@data[,paste(crops,'difference',sep="_")], key = "stat",value="value")
difference$stat<-factor(difference$stat, levels=paste(crops,'difference',sep="_"),ordered=TRUE)

interval<-gather(china@data[,paste(crops,'interval',sep="_")], key = "stat",value="value")
interval$stat<-factor(interval$stat, levels=paste(crops,'interval',sep="_"),ordered=TRUE)

In [ ]:
### Plot the distribution of differences
dp<-ggplot(difference, aes(x=stat,y=value)) + 
  geom_violin(lwd=0.5,color="#6497b1",fill="#b3cde0")+
  geom_boxplot(width=0.05,fill="#005b96",outlier.size=0,lwd=0.3,outlier.shape=NA)+
  geom_hline(yintercept=0,col="red",linetype="dashed")+
  theme_classic()+ 
  theme(axis.title=element_text(size=7,face="plain"),
          axis.text=element_text(size=7))+
  labs(y="difference between the predicted and the observed values\nof log proportion of area under cultivation\n",x="")+
  scale_x_discrete(labels=crops)+
  scale_y_continuous(breaks=seq(-20,25,5))

# Optionally plot in jupyter as well
options(repr.plot.width=6,repr.plot.height=3.5,repr.plot.res=300)
dp

In [14]:
width=14.3
height=8.3
ggsave(dp,filename = path2difference,width = width,height = height,units = "cm",dpi = 600)

In [ ]:
### Plot the distribution of intervals

ip<-ggplot(interval, aes(x=stat,y=value)) + 
  geom_violin(lwd=0.5,color="#6497b1",fill="#b3cde0")+
  geom_boxplot(width=0.05,fill="#005b96",outlier.size=0,outlier.shape=NA)+
  theme_classic()+ 
  theme(axis.title=element_text(size=7,face="plain"),
          axis.text=element_text(size=7))+
  labs(y="prediction 90% confidence interval \nin log proportion of area under cultivation\n",x="")+
  scale_x_discrete(labels=crops)+
  scale_y_continuous(breaks=seq(0,20,2))

# Optionally plot in jupyter as well
options(repr.plot.width=6,repr.plot.height=3.5,repr.plot.res=300)
ip

In [18]:
width=14.3
height=8.3
ggsave(ip,filename = path2intervals,width = width,height = height,units = "cm",dpi = 600)

In [16]:
# Plot the potential nich predictions and diagnostics for the present distribution
# Define the labels:
    columns<-paste(crop,c("logArea","q95","mean","q5","difference","interval"),sep="_")
    labels<-c("A: OBSERVED VALUES", "B: PREDICTIONS AT 95TH PERCENTILE","C: MEAN PREDICTIONS","D: PREDICTIONS AT 5TH PERCENTILE","E: DIFFERENCE BETWEEN THE MEAN\nPREDICTIONS AND THE OBSERVED VALEUS\n","F: SIZE OF THE 90% CONFIDENCE INTERVAL\n")

In [ ]:
# Plot the potential nich predictions and diagnostics for the present distribution on the maps for each crop

for (crop in crops){
    print(crop)
    
    check_maps<-tm_shape(china) +
        tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.8)+
        # Define the colors of the polygons and the legend
        tm_fill(col=columns[1:4], title.col="Period",style="cont",n=20,legend.reverse = TRUE,palette="-RdYlBu",title = "log area\nfraction\nunder\ncultivation",showNA = F,midpoint=NA)+
        # Define the facets to draw:
        tm_facets(free.scales=FALSE,free.coords=FALSE,ncol = 2,nrow = 2)+   
        # Add the shape of the predicted niche  
        #tm_shape(inner_border_check)+
        #tm_borders(col="grey1",lwd=0.8,lty="dashed")+
        #tm_facets(by = "id",free.scales=FALSE,free.coords=FALSE,ncol=2,nrow=2)+
        # Add the outer border to the plot
        tm_shape(outer_border)+
        tm_borders(col="grey75",lwd=0.8)+
        theme_tmaps+
        # Format layout:
        tm_layout(panel.labels=labels[1:4],
              legend.outside.size=0.08,
              legend.outside = TRUE,
              legend.text.size = 0.5,
              panel.label.size = 0.8,
              legend.title.size=0.6,
              panel.label.height=2,
              inner.margins=c(0,0,0,0))


    # Make difference maps:
    diff_maps<-tm_shape(china)+
        tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.6)+
        tm_fill(col=columns[5:6], title.col="Period",style = "cont",legend.reverse = TRUE,palette=c("-RdBu"),title = c("log area\nfraction\nunder\ncultivation"),showNA = F)+
        tm_shape(outer_border)+
        tm_borders(col="grey75",lwd=0.35)+    
        theme_tmaps+
        tm_layout(panel.labels=labels[5:6],
              legend.outside.size=0.08,
              legend.outside = TRUE,
              legend.text.size = 0.4,
              panel.label.size = 0.55,
              legend.title.size=0.6,
              panel.label.height=3,
             inner.margins=c(0,0,0,0),
             outer.margins=c(0.01,0.0425,0.01,0.02),
             between.margin=1)+
    tm_facets(free.scales=FALSE,free.coords=FALSE,nrow=1,ncol=2)+
    tm_credits("Coordinate system: World Geodetic System 1984",position=c("left","BOTTOM"),size=0.65)


    cm<-tmap_arrange(check_maps, diff_maps, heights=c(19.45/30,10.55/30),outer.margins=NULL)
    
    ### Save maps
    width=14.3 #17.4
    height=16 #20.4
    tmap_save(tm = cm,filename = paste(path2outputs,"/",crop,"_predictive_check_maps.png",sep=""),width = width,height = height,units = "cm",dpi = 600)
    
    }

## Predictions

In [47]:
pred_maps<-tm_shape(china) +
    # Define the grid form maps drawing
    tm_graticules(lwd = 0.05,col="grey75",n.x=2,n.y=3,labels.size = 0.8)+
    # Define the colors of the polygons and the legend
    tm_fill(col=columns, title.col="Period",style="cont",breaks=seq(-26,6,2),legend.reverse = TRUE,palette="-RdYlBu",title = "log area\nfraction\nunder\ncultivation",showNA = F,midpoint=NA)+
    # Define the facets to draw:
    tm_facets(free.scales=FALSE,free.coords=FALSE,ncol = 2,nrow = 5)+   
     # Add the shape of the predicted mean niche  
    #tm_shape(inner_border)+
    #tm_borders(col="grey10",lwd=1,lty="dashed")+
    #tm_facets(by = "id",free.scales=FALSE,free.coords=FALSE,ncol=2,nrow=4)+
    # Add the outer border to the plot
    tm_shape(outer_border)+
    tm_borders(col="grey75",lwd=1)+
    # Add the labels of the relevant regions
    #tm_shape(reg)+
    #tm_text("label",size=1.5)+
    #tm_facets(by = "period",free.scales=FALSE,free.coords=FALSE,ncol=2,nrow=4,drop.empty.facets = FALSE)+
    # Add the location of macro and microfossils to the map
    #tm_shape(macros)+
    #tm_symbols(col ="Sample_type", shape="Sample_type",size=0.6,legend.col.show=FALSE,legend.shape.show=FALSE,palette=c("mediumorchid4","red","mediumseagreen","slateblue1"),shapes = c(24,25,21,22),alpha=0.6)+
    #tm_text(text="map_labels",size=0.8,ymod="adjy",xmod="adjx")+
    #tm_facets(by = "period",free.scales=FALSE,free.coords=FALSE,ncol=2,nrow=4,drop.empty.facets = FALSE)+  
    # Format layout:
    tm_layout(panel.labels=facets,
              panel.label.bg.color="white",
              frame=FALSE,
              legend.outside.size=0.08,
              legend.outside = TRUE,
              legend.text.size = 0.4,
              panel.label.size = 1,
              panel.label.fontface = "bold",
              legend.title.size=1,
              frame.lwd = NA,
              panel.label.height=2,
              attr.outside=TRUE
             )+
    # Add legend for symbols
    #tm_add_legend("symbol",shape=c(17,24,17,21,17,22,17,25,17,17,65,17,17,66,17), col = c("white","mediumorchid4","white","mediumseagreen","white","slateblue1","white","red","white","white","black","white","white","black","white"), size = c(0.8,0.8,0.8,0.8),labels = str_wrap(c("","charred seeds","","pollen","","starch","","F. tataricum charred seeds","","","inferred origin","","","maximum suitability area",""),width=12),alpha=0.6)+
    # Add projection information
    tm_credits("\n\nCoordinate system: World Geodetic System 1984",position=c("left","bottom"))

In [ ]:
width=14.3
height=23.2
tmap_save(tm = pred_maps,filename = path2predMaps,width = width,height = height,units = "cm",dpi = 600)